In [ ]:
# refereces https://medium.com/we-talk-data/how-can-i-use-knn-and-random-forest-models-in-pytorch-6083f5ef370a



import torch 
from torch.utils.data import DataLoader , TensorDataset
from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split

data = load_iris()
X,y   = data.data, data,target 

device = torch.device("cude" if torch.cuda.isavailable() else "cpu")

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train,y_train), batch_size=16, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test, batch_size=16))

class KNNClassifier:
    def __init__(self,k=3):
        self.k = k
        self.X_train = None 
        self.y_train = None 

    def fit(self,X_train, y_train):
        self.X_train = X_train # store training data for distance calculation
        self.y_train = y_train

    def predict(self, X):
        distances = torch.cdist(X,self.X_train) #calculate eucliedean distance 
        knn_indices = distances.topk(self.k, largest =False).indices #get indices
        knn_labels = self.y_train[knn_indices] # reterieve labels of k nearest neighbors
        predictions = torch.mode(knn_labels, dim=1).values # majority voting : find most common label in neghbors
        return predictions

knn = KNNClassifier(k=3) # initialise the train model
knn.fit(X_train, y_train)
predictions = knn.predict(X_test) # predict om test set

accuracy = (predictions == y_test).float().mean() #calcualte accuracy
print(f'Test Accuracy {accuracy:.4f}')
